# Stop detector

Yeah karena datanya sangat banyak stay nya mari kita hempaskan mereka

In [1]:
def create_plot(data):
    # Ambil baris pertama dari DataFrame
    first_row = data.iloc[0]

    # Ambil nilai latitude dan longitude dari baris pertama
    latitude = first_row['latitude']
    longitude = first_row['longitude']
    
    m = folium.Map(location=[latitude, longitude], zoom_start=25)
    
    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
            popup=f"User ID: {row['maid']}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}",
        ).add_to(m)
    
    return m

In [2]:
import folium
from shapely.geometry import Point, LineString

def create_plot_line(data):
    # Buat peta dengan lokasi awal berdasarkan rata-rata koordinat dari data linestring
    linestring_avg_lat = data["geometry"].apply(lambda x: x.centroid.y).mean()
    linestring_avg_lon = data["geometry"].apply(lambda x: x.centroid.x).mean()
    m = folium.Map(location=[linestring_avg_lat, linestring_avg_lon], zoom_start=15)

    # Tambahkan polyline ke peta menggunakan titik-titik dari linestring
    for linestring in data["geometry"]:
        folium.PolyLine(locations=list(linestring.coords), color='blue', weight=5, opacity=0.7).add_to(m)

    # Tambahkan CircleMarkers untuk setiap titik
    for index, row in data.iterrows():
        for point in row["geometry"].coords:
            folium.CircleMarker(
                location=[point[1], point[0]],  # Urutan koordinat diubah karena folium menggunakan (lat, lon)
                radius=5,  # Ukuran marker
                color="blue",  # Warna marker
                fill=True,
                fill_color="blue",  # Warna pengisian marker
                fill_opacity=0.7,  # Opasitas pengisian marker
                popup=f"User ID: {row['maid']}<br>Latitude: {point[1]}<br>Longitude: {point[0]}",
            ).add_to(m)

    return m

In [3]:
from shapely.geometry import LineString

def create_line(data):
    data = data.to_crs('EPSG:4326')
    data = data['geometry']
    # Buat peta dengan lokasi awal berdasarkan rata-rata koordinat dari data linestring
    avg_lat = data.apply(lambda x: x.centroid.y).mean()
    avg_lon = data.apply(lambda x: x.centroid.x).mean()
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=15)

    # Tambahkan polyline untuk setiap linestring
    for linestring in data:
        coordinates = [(lat, lon) for lon, lat in linestring.coords]
        folium.PolyLine(locations=coordinates, color='purple', weight=8, opacity=0.7).add_to(m)

    # Tambahkan GeoJson dari data GeoPandas
    folium.GeoJson(data.to_json(), name='Garis Jalan').add_to(m)

    return m

In [4]:
def bar_chart_traj(data):
    data = data.copy()
    df = pd.DataFrame(data)
    
    if 'tanggal' in df.columns:
        # Menghitung jumlah data berdasarkan tanggal
        jumlah_data_per_tanggal = df.groupby('tanggal').size()
    
    else:
        # Ambil tanggal dari kolom 'start_t' dan tambahkan sebagai kolom baru 'tanggal'
        df['tanggal'] = df['start_t'].dt.date
    
        # Menghitung jumlah data berdasarkan tanggal
        jumlah_data_per_tanggal = df.groupby('tanggal').size()
        
    # Plot bar chart
    plt.figure(figsize=(10, 6))
    jumlah_data_per_tanggal.plot(kind='bar', color='skyblue')
    plt.title(f'Jumlah Data per Tanggal: {jumlah_data_per_tanggal.sum()}')
    plt.xlabel('Tanggal')
    plt.ylabel('Jumlah Data Trajectory')
    plt.xticks(rotation=75)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [5]:
def bar_chart_maid(data):
    data = data.copy()
    df = pd.DataFrame(data)
    
    if 'tanggal' in df.columns:
        # Menghitung jumlah maid berdasarkan tanggal
        jumlah_maid_unik_per_hari = df.groupby('tanggal')['maid'].nunique()
    
    else:
        # Ambil tanggal dari kolom 'start_t' dan tambahkan sebagai kolom baru 'tanggal'
        df['tanggal'] = df['start_t'].dt.date
        
        # Menghitung jumlah maid berdasarkan tanggal
        jumlah_maid_unik_per_hari = df.groupby('tanggal')['maid'].nunique()


    # Plotting
    plt.figure(figsize=(10, 6))
    jumlah_maid_unik_per_hari.plot(kind='bar', color='skyblue')
    plt.title(f'Total Jumlah Maid Unik per Hari: {jumlah_maid_unik_per_hari.sum()}')
    plt.xlabel('Tanggal')
    plt.ylabel('Jumlah Maid Unik')
    plt.xticks(rotation=75)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [6]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas 
import matplotlib.pyplot as plt
import geoviews
import folium
import mapclassify

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts

import warnings
warnings.filterwarnings('ignore')

plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=500, frame_height=400))

mpd.show_versions()


MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
executable : /raid/arkham/miniconda3/envs/arkham/bin/python3
machine    : Linux-4.15.0-142-generic-x86_64-with-glibc2.27

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.6.2
GDAL data dir: /raid/arkham/miniconda3/envs/arkham/lib/python3.10/site-packages/fiona/gdal_data
PROJ       : 9.3.1
PROJ data dir: /raid/arkham/miniconda3/envs/arkham/lib/python3.10/site-packages/pyproj/proj_dir/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.3
pandas     : 2.1.4
fiona      : 1.9.5
numpy      : 1.23.5
shapely    : 2.0.1
rtree      : 1.2.0
pyproj     : 3.6.1
matplotlib : 3.5.3
mapclassify: 2.6.1
geopy      : 2.4.0
holoviews  : 1.18.3
hvplot     : 0.9.2
geoviews   : 1.11.1
stonesoup  : None


In [7]:
df = pd.read_csv('../../../DataGPS_Malioboro/filter2_maljan_nov.csv')
df_grouped = df.groupby(['maid', 'tanggal'])
tdf = mpd.TrajectoryCollection(df, traj_id_col='maid', t='datetime_wib', y='latitude', x='longitude')
tdf

TrajectoryCollection with 1838 trajectories

In [8]:
#record points
len(df)

40693

In [9]:
#total maid unik per bulan
len(df.groupby(['maid']))

1942

In [ ]:
#total maid unik per hari
bar_chart_maid(df)

In [88]:
index = 0
df_1min = pd.DataFrame()

while index < len(tdf):
    my_traj = tdf.trajectories[index]
    split = mpd.ObservationGapSplitter(my_traj).split(gap=timedelta(minutes=5))
    if len(split) > 1:
        split_df = split.to_traj_gdf()
        df_1min = df_1min._append(split_df, ignore_index=True)
    index += 1

In [89]:
df_1min['maid'] = df_1min['maid'].apply(lambda x: x.split('_')[0])

In [90]:
# total maid unik per sebulan
df_1min['maid'].nunique()

767

In [ ]:
# total maid unik per hari
bar_chart_maid(df_1min)

In [ ]:
# total trajectory
bar_chart_traj(df_1min)

In [91]:
# apply for traj length more than 0

df_1min_length = df_1min[df_1min['length'] > 0].reset_index(drop=True)
df_1min_length= df_1min_length.sort_values(by='length', ascending=False).reset_index(drop=True)
# df_1min_length = df_1min_length.to_crs('EPSG:3857')
# df_1min_length['length_meter'] = df_1min_length['geometry'].length

In [92]:
# total maid unik per sebulan
df_1min_length['maid'].nunique()

724

In [ ]:
# total maid unik per hari
bar_chart_maid(df_1min_length)

In [ ]:
# total trajectory
bar_chart_traj(df_1min_length)

In [93]:
# apply for traj length >5 
df_1min_length_5met = df_1min_length[df_1min_length['length']>5]

In [94]:
# total maid unik per sebulan
df_1min_length_5met['maid'].nunique()

420

In [ ]:
# total maid unik per hari
bar_chart_maid(df_1min_length_5met)

In [ ]:
# total trajectory
bar_chart_traj(df_1min_length_5met)

## Create points gdf

In [95]:
index = 0
points_1min = pd.DataFrame()

while index < len(tdf):
    my_traj = tdf.trajectories[index]
    split = mpd.ObservationGapSplitter(my_traj).split(gap=timedelta(minutes=5))
    if len(split) > 1:
        points_df = split.to_point_gdf()
        points_1min = points_1min._append(points_df)
    index += 1
points_1min.reset_index(drop=False, inplace=True)
points_1min['maid'] = points_1min['maid'].apply(lambda x: x.split('_')[0])

In [96]:
def create_points(points, traj):
    traj = traj.copy()
    points = points.copy()
    filtered_data = []
        
    for index, row in traj.iterrows():
        maid = row['maid']
        start_t = row['start_t']
        end_t = row['end_t']
        
        # Filter points from data based on maid and datetime range
        filtered = points[(points['maid'] == maid) & 
                          (points['datetime_wib'] >= start_t) & 
                          (points['datetime_wib'] <= end_t)]
        
        filtered_data.append(filtered)

    # Concatenate all filtered dataframes into one
    df_filtered = pd.concat(filtered_data, ignore_index=True)
    
    return df_filtered 

In [97]:
dfp_1min_length_5met = create_points(points_1min, df_1min_length_5met)

In [98]:
len(df[df['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688'])
a = df[df['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688']

In [101]:
df_1min_length[df_1min_length['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688']

maid             start_t  \
427   b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 14:23:32   
511   b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 18:04:08   
541   b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 17:48:08   
933   b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 15:49:37   
1381  b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-13 15:33:08   
1432  b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 16:55:30   
1641  b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 17:17:02   
2241  b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 17:38:11   
3573  b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-13 13:01:07   
4454  b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-13 13:56:08   

                   end_t                                           geometry  \
427  2021-11-01 14:28:50  LINESTRING (110.36550 -7.79426, 110.36550 -7.7...   
511  2021-11-01 18:09:43  LINESTRING (110.36597 -7.79425, 110.36588 -7.7...   
541  2021-11-01 17:57:47  LINESTRING (110.36591 -7.79419, 110.36592 -7.7...   
933  2021-11-01 15:54:40  LINESTRING (110.36548 -7.79403, 110.36547 -7.7...   
1381 2021-11-13 15:38:21  LINESTRING (110.36549 -7.79419, 110.36549 -7.7...   
1432 2021-11-01 16:56:46  LINESTRING (110.36595 -7.79429, 110.36596 -7.7...   
1641 2021-11-01 17:19:41  LINESTRING (110.36591 -7.79436, 110.36591 -7.7...   
2241 2021-11-01 17:39:23  LINESTRING (110.36593 -7.79439, 110.36593 -7.7...   
3573 2021-11-13 13:01:57  LINESTRING (110.36595 -7.79254, 110.36596 -7.7...   
4454 2021-11-13 13:56:58  LINESTRING (110.36562 -7.79476, 110.36562 -7.7...   

         length   direction  
427   54.963221   91.353331  
511   40.564404  212.939462  
541   36.715319  173.417005  
933   14.557552    5.142031  
1381   6.856871    0.000000  
1432   6.351449  319.973810  
1641   5.004369    0.000000  
2241   2.803691    0.000000  
3573   1.389697   37.085420  
4454   0.831764  262.309146

In [100]:
df_1min_length_5met[df_1min_length_5met['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688']

maid             start_t  \
427   b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 14:23:32   
511   b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 18:04:08   
541   b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 17:48:08   
933   b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 15:49:37   
1381  b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-13 15:33:08   
1432  b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 16:55:30   
1641  b087a3ad-86ab-421b-bdd9-ac52382bd688 2021-11-01 17:17:02   

                   end_t                                           geometry  \
427  2021-11-01 14:28:50  LINESTRING (110.36550 -7.79426, 110.36550 -7.7...   
511  2021-11-01 18:09:43  LINESTRING (110.36597 -7.79425, 110.36588 -7.7...   
541  2021-11-01 17:57:47  LINESTRING (110.36591 -7.79419, 110.36592 -7.7...   
933  2021-11-01 15:54:40  LINESTRING (110.36548 -7.79403, 110.36547 -7.7...   
1381 2021-11-13 15:38:21  LINESTRING (110.36549 -7.79419, 110.36549 -7.7...   
1432 2021-11-01 16:56:46  LINESTRING (110.36595 -7.79429, 110.36596 -7.7...   
1641 2021-11-01 17:19:41  LINESTRING (110.36591 -7.79436, 110.36591 -7.7...   

         length   direction  
427   54.963221   91.353331  
511   40.564404  212.939462  
541   36.715319  173.417005  
933   14.557552    5.142031  
1381   6.856871    0.000000  
1432   6.351449  319.973810  
1641   5.004369    0.000000

In [99]:
len(dfp_1min_length_5met[dfp_1min_length_5met['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688'])
b = dfp_1min_length_5met[dfp_1min_length_5met['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688']

## Stop detection

In [28]:
tc = mpd.TrajectoryCollection(dfp_1min_length_5met, 'maid', t='datetime_wib')

In [29]:
split = mpd.StopSplitter(tc).split(min_duration=timedelta(seconds=60), max_diameter=100)

In [30]:
split

TrajectoryCollection with 316 trajectories

In [43]:
point_split = split.to_point_gdf()
point_split.reset_index(drop=False, inplace=True)
point_split

datetime_wib                                               maid  \
0    2021-11-08 15:07:56  07aa8f8c-9091-46ff-be83-a3f45de94fa1_2021-11-0...   
1    2021-11-09 07:31:51  07aa8f8c-9091-46ff-be83-a3f45de94fa1_2021-11-0...   
2    2021-11-09 07:35:10  07aa8f8c-9091-46ff-be83-a3f45de94fa1_2021-11-0...   
3    2021-11-09 08:16:40  07aa8f8c-9091-46ff-be83-a3f45de94fa1_2021-11-0...   
4    2021-11-01 06:39:50  0bfa1e0b-fbda-46ca-9ed9-93e5ac8a956e_2021-11-0...   
...                  ...                                                ...   
1418 2021-11-03 21:04:51  fcd76f5f-abf2-4108-8d24-e1095f0de7c6_2021-11-0...   
1419 2021-11-21 15:16:00  fd32f663-c845-4dfd-8172-f9061b7f4ee4_2021-11-2...   
1420 2021-11-21 15:16:07  fd32f663-c845-4dfd-8172-f9061b7f4ee4_2021-11-2...   
1421 2021-11-21 15:16:55  fd32f663-c845-4dfd-8172-f9061b7f4ee4_2021-11-2...   
1422 2021-11-21 15:16:57  fd32f663-c845-4dfd-8172-f9061b7f4ee4_2021-11-2...   

                        geometry           Kelurahan/Desa     Kecamatan  \
0     POINT (110.36514 -7.79866)       Kelurahan Ngupasan    Gondomanan   
1     POINT (110.36546 -7.79470)  Kelurahan Sosromenduran  Gedongtengen   
2     POINT (110.36546 -7.79470)  Kelurahan Sosromenduran  Gedongtengen   
3     POINT (110.36497 -7.79895)       Kelurahan Ngupasan    Gondomanan   
4     POINT (110.36613 -7.79275)    Kelurahan Suryatmajan     Danurejan   
...                          ...                      ...           ...   
1418  POINT (110.36611 -7.79128)    Kelurahan Suryatmajan     Danurejan   
1419  POINT (110.36561 -7.79565)    Kelurahan Suryatmajan     Danurejan   
1420  POINT (110.36561 -7.79563)    Kelurahan Suryatmajan     Danurejan   
1421  POINT (110.36562 -7.79566)    Kelurahan Suryatmajan     Danurejan   
1422  POINT (110.36562 -7.79564)    Kelurahan Suryatmajan     Danurejan   

            Kabupaten     tanggal  
0     Kota Yogyakarta  2021-11-08  
1     Kota Yogyakarta  2021-11-09  
2     Kota Yogyakarta  2021-11-09  
3     Kota Yogyakarta  2021-11-09  
4     Kota Yogyakarta  2021-11-01  
...               ...         ...  
1418  Kota Yogyakarta  2021-11-03  
1419  Kota Yogyakarta  2021-11-21  
1420  Kota Yogyakarta  2021-11-21  
1421  Kota Yogyakarta  2021-11-21  
1422  Kota Yogyakarta  2021-11-21  

[1423 rows x 7 columns]

In [44]:
c = point_split[point_split['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688']

In [ ]:
split.trajectories[0]

In [ ]:
# single trajectory
my_traj = split.trajectories[0]
traj_plot = my_traj.hvplot(title='Trajectory {}'.format(my_traj.id), line_width=7.0, tiles='CartoLight', color='slategray') 
traj_plot

In [ ]:
detector = mpd.TrajectoryStopDetector(my_traj)

In [ ]:
# Stop duration
stop_time_ranges = detector.get_stop_time_ranges(min_duration=timedelta(seconds=60), max_diameter=100)
for x in stop_time_ranges: 
    print(x)

In [ ]:
# stop points
stop_points = detector.get_stop_points(min_duration=timedelta(seconds=60), max_diameter=100)
stop_points

In [ ]:
# stop_point_plot = traj_plot * stop_points.hvplot(geo=True, size='duration_s', color='deeppink')
# stop_point_plot

In [ ]:
# Stop segments
stops = detector.get_stop_segments(min_duration=timedelta(seconds=60), max_diameter=100)
stops

In [ ]:
# stop_segment_plot = stop_point_plot * stops.hvplot(line_width=7.0, tiles=None, color='orange') 
# stop_segment_plot

## Stop detection for TrajectoryCollections

In [ ]:
detector = mpd.TrajectoryStopDetector(split)
stop_points = detector.get_stop_points(min_duration=timedelta(seconds=120), max_diameter=100)
len(stop_points)

In [ ]:
ax = tc.plot(figsize=(7,7))
stop_points.plot(ax=ax, color='red')

In [ ]:
tc = mpd.TrajectoryCollection(df_1min_length_5met, 'maid', 't')
tc

In [ ]:
type(df_1min_length_5met)

In [ ]:
df_1min_length_5met

## Map visualization

In [75]:
a[:50].reset_index()

index                                  maid  latitude   longitude  \
0      50  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794260  110.365501   
1      51  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794250  110.365501   
2      52  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794258  110.365501   
3      53  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794260  110.365501   
4      54  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794270  110.365501   
5      55  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.793878  110.365501   
6      56  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794030  110.365479   
7      57  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794040  110.365471   
8      58  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794010  110.365479   
9      59  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794020  110.365471   
10   1319  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794170  110.365494   
11   1320  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794180  110.365494   
12  10003  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794260  110.365936   
13  10004  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794270  110.365929   
14  10291  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794280  110.365959   
15  10294  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794290  110.365936   
16  10295  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794270  110.365936   
17  10313  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794176  110.365921   
18  10314  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794321  110.365929   
19  10333  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794250  110.365967   
20  10337  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794440  110.365883   
21  10338  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794430  110.365868   
22  10342  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794310  110.365913   
23  10343  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794320  110.365921   
24  12987  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.792540  110.365952   
25  12989  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.792530  110.365959   
26  12995  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794761  110.365616   
27  20383  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794170  110.365494   
28  23792  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794390  110.365929   
29  23805  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794360  110.365929   
30  23806  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794348  110.365936   
31  23807  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794320  110.365913   
32  23808  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794321  110.365929   
33  25672  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794760  110.365623   
34  29203  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.793946  110.365486   
35  30347  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794288  110.365952   
36  30359  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794358  110.365906   
37  30360  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794358  110.365906   
38  30361  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794370  110.365898   
39  30362  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794360  110.365898   
40  30363  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794350  110.365898   
41  30365  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794358  110.365906   
42  30370  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794390  110.365929   
43  30371  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794381  110.365936   
44  30372  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794380  110.365936   
45  30373  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794370  110.365929   
46  30384  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794190  110.365913   
47  30386  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794176  110.365921   
48  30389  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794360  110.365929   
49  30390  b087a3ad-86ab-421b-bdd9-ac52382bd688 -7.794350  110.365936   

           datetime_wib                             geometry  \
0   2021-11-01 14:23:32   POINT (110.3655014038086 -7.79426)   
1   2021-11-01 14:23:41   POINT (110.3655014038086 -7.79425)   
2   2021-11-01 14:26:25  POINT (110.3655014038086 -7.794258)   
3   2021-11-01 14:27:08 

In [87]:
create_plot(a[24:27])

In [52]:
create_plot_line(b)

In [ ]:
create_line(df_1min_length_sorted[df_1min_length_sorted['start_t'] > '2021-12-22'])

In [ ]:
bar_chart_traj(df_1min_length_sorted)

In [ ]:
bar_chart_maid(df_1min_length_sorted)

In [ ]:
len(df_1min_length_sorted)